In [1]:
import config as cfg
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(device + " is available")

# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the pre-trained model for sequence classification with the number of labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(cfg.le.classes_))
model=model.to(device)

NameError: name 'train' is not defined

In [14]:
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load data
train = cfg.train
test = cfg.test

# Assume the second column is the label and the first column is the text
train_texts = train[0].tolist()
test_texts = test[0].tolist()

# Convert labels to integer (if they are not already)
# This might involve using a LabelEncoder as you have categorical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = cfg.le
num_labels = len(cfg.le.classes_)

# Tokenize the text and create datasets
max_length = 256  # You might need to adjust this based on your dataset
train_dataset = IntentDataset(train_texts, cfg.train_labels, tokenizer, max_length)
test_dataset = IntentDataset(test_texts, cfg.test_labels, tokenizer, max_length)

In [16]:
import mlflow
import mlflow.pytorch


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',  
    logging_strategy="steps",  # or "epoch"
    logging_steps=50  # Log every 10 steps# directory for storing logs,
)
from transformers import TrainerCallback

class MLflowLoggingCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        # Log metrics with MLflow here
        if metrics:
            for key, value in metrics.items():
                mlflow.log_metric(key, value, step=state.global_step)

try:
    mlflow.create_experiment("BertIntentClassification Pretrained")
    mlflow(pytorch=True)
    mlflow.start_run()
    mlflow.log_param("epochs", training_args.num_train_epochs)
    mlflow.log_param("batch_size", training_args.per_device_train_batch_size)
    mlflow.log_param("learning_rate", training_args.learning_rate)
    mlflow.log_param("weight_decay", training_args.weight_decay)
    mlflow.log_param("warmup_steps", training_args.warmup_steps)
    mlflow.log_param("max_length", max_length)
    mlflow.log_param("num_labels", num_labels)
    mlflow.log_param("model", "bert-base-uncased")

except:
    pass
#mlflow.log_params(your_params_dict)  # Log any initial parameters
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[MLflowLoggingCallback()]
)
trainer.train()
trainer.evaluate()


SyntaxError: invalid syntax (4143003355.py, line 28)

In [9]:
trainer.save_model('results/final_bert_evaluated')

In [11]:
trainer.evaluate()

{'eval_loss': 0.1844148188829422,
 'eval_runtime': 1.1111,
 'eval_samples_per_second': 765.03,
 'eval_steps_per_second': 12.601,
 'epoch': 3.0}

ModuleNotFoundError: No module named 'datasets'